In [ ]:
import mysql.connector
import pandas as pd
import datetime

In [ ]:
def connect_to_db():
    
    print('Connecting to Master Database..')
    
    try:
        mydb = mysql.connector.connect(
          host="xxx.xxx.com",
          user="xxx",
          password="xxx",
          database="pr_master_db",
          port=xxx
        )
        
    except mysql.connector.Error as error:
        print("Failed to connect to MySQL database: {}".format(error))
    else:
        print('Succefully connected to Db!')
        return mydb

In [ ]:
def define_query():
    
	
	query = f"""SELECT requests.date, products_info.id as PID, products_info.rel_ean, requests.shop_id,  competitors.name, g.url, tracks.created_at
		from requests
		INNER join competitors ON competitors.id= requests.shop_id and competitors.active = 1
		INNER JOIN products_info ON products_info.rel_sku = requests.identity and products_info.shop_id = requests.shop_id
		INNER JOIN tracks ON tracks.product_id = products_info.id
								AND tracks.selector_id =(select selector.id from selector WHERE selector.competitor_id= requests.shop_id)
		left join tracks as g on g.product_id = products_info.id and g.selector_id = (
			select selector.id from selector inner join competitors on competitors.id = selector.competitor_id and competitors.`name` = concat('google.', '{country_code}')
		)
		where requests.shop_id in (
							select competitors.id
							from competitors
							inner join selector on selector.competitor_id = competitors.id
							inner join selector_country_codes on selector_country_codes.id = selector.country_code  and  selector_country_codes.country_code = '{country_code}'
							where competitors.active = 1
		)
		AND (week(requests.date) = WEEK(NOW()) OR week(requests.date) = WEEK(NOW())-1)
		AND year(requests.date) = YEAR(NOW())
		AND rel_ean NOT LIKE '20000%'
		AND rel_ean NOT LIKE '24000%'
		AND rel_ean > 0
		AND ROUND(DATEDIFF(requests.date,tracks.created_at)/30,0)<=6"""
    
	return query

In [ ]:
def run_query(query,mydb):
    
    # create a cursor object
    mycursor = mydb.cursor()

    # execute a SQL query
    mycursor.execute(query)
    # fetch the results
    results = mycursor.fetchall()
    mydb.close()
    
    print(f'\nFetched Data at: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

    df = pd.DataFrame(results)
    df.columns = mycursor.column_names   
    print(f'Total Rows: {df.shape[0] - 1}')
        

    return df

In [ ]:
def export_csv(df):
    
    df.to_csv('fetced_data.csv', encoding='utf-8', index=False)
    print('\nData exported to file named: fetced_data.csv\n\n')


In [ ]:
def main():    
    mydb = connect_to_db()
    df = run_query(define_query(),mydb)
    export_csv(df)

In [ ]:
if __name__ == "__main__":
    main()